In [10]:
from compartmentsBase import *
from sympy import *
init_printing()


# temp helpers 
def display_hc(k,g,pi,name='c'):
    lhs = symbols('k_' + name)
    rhs = k
    display(Eq(lhs,rhs,evaluate=False))

    lhs = symbols('g_' + name)
    rhs = g
    display(Eq(lhs,rhs,evaluate=False))

    lhs = symbols('\pi_' + name)
    rhs = pi.expr
    display(Eq(lhs,rhs,evaluate=False))


$$
\def\n{\mathbf{n}}
\def\x{\mathbf{x}}
\def\N{\mathbb{\mathbb{N}}}
\def\X{\mathbb{X}}
\def\NX{\mathbb{\N_0^\X}}
\def\C{\mathcal{C}}
\def\Jc{\mathcal{J}_c}
\def\DM{\Delta M_{c,j}}
\newcommand\diff{\mathop{}\!\mathrm{d}}
\def\Xc{\mathbf{X}_c}
\newcommand{\muset}[1]{\dot{\{}#1\dot{\}}}
$$


For a compartment population $\n \in \NX$ evcolving stochastically according to stoichiometric equations from transition classes $\C$, we want to find an expression for
$$
\frac{\diff}{\diff t}\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>
$$
in terms of expectations of population moments $M^\alpha, M^{\beta}, \ldots$

In [11]:
from compartmentsB import pi_c_identity, pi_c_poisson

D = 1 # number of species

y = ContentVar('y')
x = ContentVar('x')

# Intake
transition_I = Transition(EmptySet(), Compartment(y))
k_I = Constant('k_I')
g_I = 1
pi_I = pi_c_poisson(
    Symbol("\pi_I", positive=True),
    y[0],
    Symbol("\lambda", positive=True))

# Exit
transition_E = Transition(Compartment(x), EmptySet())
k_E = Constant('k_E')
g_E = 1
pi_E = pi_c_identity()

# birth
transition_b = Transition(Compartment(x), Compartment(x + ContentChange(1)))
k_b = Constant('k_b')
g_b = 1
pi_b = pi_c_identity()

# death
transition_d = Transition(Compartment(x), Compartment(x + ContentChange(-1)))
k_d = Constant('k_d')
g_d = x[0] # TODO x should be enough here, in case D=1.
pi_d = pi_c_identity()

transitions = [
    (transition_I, k_I, g_I, pi_I),
    (transition_E, k_E, g_E, pi_E),
    (transition_b, k_b, g_b, pi_b),
    (transition_d, k_d, g_d, pi_d)
]

# display(transition_I)
# display_hc(k_I, g_I, pi_I, name='I')
# display(transition_E)
# display_hc(k_E, g_E, pi_E, name='E')
# display(transition_b)
# display_hc(k_b, g_b, pi_b, name='b')
# display(transition_d)
# display_hc(k_d, g_d, pi_d, name='d')

### (1)
From the definition of the compartment dynamics, we have
$$
\diff M^\gamma = \sum_{c \in \C} \sum_{j \in \Jc} \DM^\gamma \diff R_{c,j}
$$
We apply Ito's rule to derive
$$
\diff f(M^\gamma, M^{\gamma'}, \ldots) = \sum_{c \in \C} \sum_{j \in \Jc}
    \left(
        f(M^\gamma + \DM^\gamma, M^{\gamma'} + \DM^{\gamma'}, \ldots)
        - f(M^\gamma, M^{\gamma'}, \ldots)
    \right) \diff R_{c,j}
$$

Assume, that $f(M^\gamma, M^{\gamma'}, \ldots)$ is a polynomial in $M^{\gamma^i}$ with $\gamma^i \in \N_0^D$.

Then $\diff f(M^\gamma, M^{\gamma'}, \ldots)$ is a polynomial in $M^{\gamma^k}, \DM^{\gamma^l}$ with $\gamma^k, \gamma^l \in \N_0^D$, that is,
$$
\diff f(M^\gamma, M^{\gamma'}, \ldots) = \sum_{c \in \C} \sum_{j \in \Jc}
    \sum_{q=1}^{n_q} Q_q(M^{\gamma^k}, \DM^{\gamma^l})
    \diff R_{c,j}
$$
where $Q_q(M^{\gamma^k}, \DM^{\gamma^l})$ are monomials in $M^{\gamma^k}, \DM^{\gamma^l}$.

### (2)
Let's write $Q_q(M^{\gamma^k}, \DM^{\gamma^l})$ as
$$
Q_q(M^{\gamma^k}, \DM^{\gamma^l}) = k_q \cdot \Pi M^{\gamma^k} \cdot \Pi M^{\gamma^k}
$$
where $k_q$ is a constant,
  $\Pi M^{\gamma^k}$ is a product of powers of $M^{\gamma^k}$, and
  $\Pi \DM^{\gamma^l}$ is a product of powers of $\DM^{\gamma^l}$.
  
Analogous to the derivation in SI Appendix S.3, we arrive at the expected moment dynamics
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t} =
    \sum_{c \in \C} \sum_{q=1}^{n_q} \left<
        \sum_{j \in \Jc} k_q \cdot \Pi M^{\gamma^k} \cdot \Pi \DM^{\gamma^k} \cdot h_{c,j}(\n)
    \right>
$$

### (3)
Analogous to SI Appendix S.4, the contribution of class $c$, monomial $q$ to the expected dynamics of $f(M^\gamma, M^{\gamma'}, \ldots)$ is
$$
\begin{align}
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
    &= \left<
        {\large\sum_{j \in \Jc}} k_q \cdot \Pi M^{\gamma^k} \cdot \Pi \DM^{\gamma^l} \cdot h_{c,j}(\n)
    \right>
    \\
    &= \left<
        {\large\sum_{\Xc}} w(\n; \Xc) \cdot k_c \cdot k_q \cdot \Pi M^{\gamma^k} \cdot g_c(\Xc) \cdot
        \left<
            \Pi \DM^{\gamma^l} \;\big|\; \Xc
        \right>
    \right>
\end{align}
$$


### (5)
Now let
$$
l(\n; \Xc) = k_c \cdot k_q \cdot \Pi(M^{\gamma^k}) \cdot g_c(\Xc) \cdot
        \left<
            \Pi \DM^{\gamma^l} \;\big|\; \Xc
        \right>
$$

Plugging in the concrete $\gamma^l$ and expanding, $l(\n; \Xc)$ is a polynomial in $\Xc$.

Monomials are of the form $k \x^\alpha$ or $k \x^\alpha \x'^\beta$ with $\alpha, \beta \in \N_0^D$.
(Note that occurences of $\Pi M^{\gamma^k}$ are part of the constants $k$.)

Consider again the different cases of reactant compartments $\Xc$:

(1) $\Xc = \emptyset$:
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
    = \left<l(\n)\right>
$$

(2) $\Xc = \muset{\x}$:
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
    = \left<R(l(\n; \muset{\x})\right>
$$
where $R$ replaces all $k \x^\alpha$ by $k M^\alpha$.

(3) $\Xc = \muset{\x, \x'}$:
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
    = \frac{1}{2}\left<R'(l(\n; \muset{\x, \x'})\right>
    \: - \:
    \frac{1}{2}\left<R(l(\n; \muset{\x, \x})\right>
$$
where $R'$ replaces all $k \x^\alpha \X'^\beta$ by $k M^\alpha M^\beta$,
and again $R$ replaces all $k \x^\alpha$ by $k M^\alpha$.




### (6)
Finally, sum over contributions from all $c$, $q$ for the total
$$
\frac{\diff\left< f(M^\gamma, M^{\gamma'}, \ldots) \right>}{\diff t}
$$

In [12]:
from ito import *
from compartmentsB import getCompartments, deltaM, subsDeltaM, get_dfMdt_contrib

def get_dfMdt(transitions, fM, D):
    dfM = ito(fM)
    monomials = decomposeMomentsPolynomial(dfM)
    contrib = list()
    for c, (transition, k_c, g_c, pi_c) in enumerate(transitions):
        for q, (k_q, pM, pDM) in enumerate([(m.k, m.pM, m.pDM) for m in monomials]):
            reactants = getCompartments(transition.lhs)
            products = getCompartments(transition.rhs)
            DM_cj = deltaM(reactants, products, D)
            pDMcj = subsDeltaM(pDM, DM_cj)
            cexp = pi_c.conditional_expectation(pDMcj)
            l_n_Xc = k_c * k_q * pM * g_c * cexp
            dfMdt = get_dfMdt_contrib(reactants, l_n_Xc, D)
            contrib.append(dfMdt)
            #print(f'c={c}, q={q}')
            #display(Eq(symbols('\\text{dfMdt}'), dfMdt))
    return Add(*contrib)

display(get_dfMdt(transitions, Moment(0), D))

-k_E⋅Moment(0) + k_I